### Solutions, First Part

This notebook contains the solutions the exercise of the First part of the workshop. To see the solutions in action *copy-paste* the code in the code block of the exercise. Notice, this code will not run as is, you have to copy-paste before running. 

#### First Exercise

In [ ]:
def whole_shuffler(M, rng):
    '''
    Your function here. Notice, we must pass the rng to ensure reproducibility
    
    '''
    shape = M.shape
    N = shape[0] * shape[1]
    M = M.flatten()
    inds = rng.choice(M, size = N, replace=False)
    # Fancy indexing
    shuffled = M[inds].reshape(shape)
    return(shuffled)
    
# Shuffle a matrix using the function you wrote
seed = 101112
rng = np.random.default_rng(seed)

to_be_shuffled = np.arange(10).reshape(5,2)
shuffled = whole_shuffler(to_be_shuffled, rng)

print(f"This is the non-shuffled matrix:\n{to_be_shuffled}\n")
print(f"This is the wholly-shuffled matrix:\n{shuffled}")

#### Second Exercise

First block:

In [ ]:
# Instantiate a rng
seed = 1122334455
rng = np.random.default_rng()

# Instantiate two non spherical bivariate normal distribution.
mvn_0 = ss.multivariate_normal(mean=np.array([3,3]), cov=np.array([[1.5,0.8],[0.8, 1.5]]))
mvn_0.random_state = rng
mvn_1 = ss.multivariate_normal(mean=np.array([-1, 2]), cov=np.array([[1.5,-0.8],[-0.8, 1.5]]))
mvn_1.random_state = rng
# Put those in a list, it will be useful to sample the variates
normals = [mvn_0, mvn_1]

# mixture for the two distributions
mixture = [0.3, 0.7]
# number of variates in the sample
N = 350

# Data generation process
# First Randomly pick one of the two normal distribution following the mixture probabilities
# ultimately we want a random vector of 0s and 1s (e.g. [0,1,1,1,0, etc.]) based on the mixture probability. 
# This is the object of our inference
selected_norm = rng.choice([0,1], size=N, p=mixture)

# Second, generate the normal variates depending on the selected_norm vector
sample = np.zeros((N,2))
for i,j in enumerate(selected_norm):
    # Notice, we are generating a random *vector*, so we will store it as a row in a matrix
    sample[i,:] = normals[j].rvs(size=1)

print(f"This is the start of our sample:\n{np.round(sample[:10,:], 3)}")

Second block:

In [ ]:
# Implement the formula above in a function
def which_norm(samp, mvnorm_0, mvnorm_1, prior=[0.5, 0.5]):
    '''
        Implement Bayes rule to calculate the probability that samp was generated by the multivariate normals
    mvnorm_0 and mvnorm_1.

    :param samp: The data point as a np.array
    :param mvnorm_0: The first multivariate normal, a scipy.stats.multivariate_normal frozen distribution
    :param mvnorm_1: The second multivariate normal, a scipy.stats.multivariate_normal frozen distribution
    :param prior: Your prior belief about which mvn generated the datapoint. A vector or list
    :return: A np.array containing the probability of samp being generated by mvnorm_0 or mvnorm_1
    '''
    
    pdf_0 = mvnorm_0 # Get the pdf of the first mvnorm at samp. Use the .pdf() method #.pdf(samp)
    pdf_1 = mvnorm_1 # Get the pdf of the second mvnorm at samp. Use the .pdf() method #.pdf(samp)
    # Write the numerator from the formula above using the prior vector.
    numerator = np.array([pdf_0*prior[0]
                          , pdf_1*prior[1]
                         ])
    denominator = numerator.sum()
    return numerator/denominator

# apply along axis
Ps = np.apply_along_axis(func1d=which_norm, 
                         arr=sample, 
                         # Which mvnorm should we pass?
                         mvnorm_0=mvn_0, 
                         mvnorm_1=mvn_1, 
                         # What is our prior? 
                         prior=[0.5,0.5],  
                         # What axis are we applying this to?
                         axis=1
                        )
print(f"These are the first 10 inferred assignemnts:\n{np.round(Ps[:10,:],3)}\n")

# Extra:
# can you reproduce the plot above with the calculate probability (Ps) instead of the real values (selected_norm)?
# Hint: Check the c argument of ax.plot() OR ax.scatter() from matplotlib
fig, ax = plt.subplots()
ax.scatter(x=sample[:,0], y=sample[:,1], c=Ps[:,1])
ax.set_aspect('equal')
_ = ax.set_title("Normal Mixture Inference")